# Libraries

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Functions

## Temperature ramp

In [ ]:
#Given the path to the file as folde and file name, it reutrns the temperature ramp as a Pandas dataframe.
#The time is in a strange format and needs to be modified depending on the format chosen during the measurments
def ReadTemperatureData(FolderTR,FileTR):
    TemperatureRamp=pd.read_csv(FolderTR+"/"+FileTR+".csv") #Read the temperature ramp data
    #give meaningful names to the dataset columns
    TemperatureRamp.rename(columns={"Date/Time": "Time", "COM9.ID001-3216.INPUT.PVInValue": "Temperature","COM9.ID001-3216.SP.SP1": "Setpoint","COM9.ID001-3216.INPUT.PVDot": "Unknown"}, inplace=True)
    return TemperatureRamp

def AdjustTime(TemperatureRamp, unit):
    #Time is in a strange format. Subtracting the first time you get the time starting from zero.
    #Then, I convert the time in hours by multplying for the unit time I select while acquiring the data that was 24h
    TimeUnit=unit #hours
    TemperatureRamp['Time']=(TemperatureRamp['Time']-TemperatureRamp['Time'][0])*TimeUnit
    return TemperatureRamp

## XPS Data

In [ ]:
#reads the XPS Data and gives meaninigful name to the columns. The colums must be 5
def ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns):
    XPS=pd.read_table(FolderTR+"/"+FileXPS, skiprows=lambda x: x%DataPerSpectrum<HeaderColumns, header=None)
    XPS.columns=["Kinetic energy", "Counts", "Initial Temperature", "Binding Energy", "CPS"]
    return XPS

#Add the temperature to the temperature column at which the acquisition started
def SetTemperatureColumn(XPS, NumberOfSpectra, regression):
    for i in range(0, NumberOfSpectra):
        XPS["Initial Temperature"][i*NumericDataPerSpectrum:(i+1)*NumericDataPerSpectrum]=regression.intercept+regression.slope
    return XPS

#Returns a vector with the temperature at which each spectrum was acquired
def TemperatureVector(regression, time, NumberOfSpectra):
    t=np.arange(NumberOfSpectra+1)
    Duration=(time[len(time)-1]-time[0])/NumberOfSpectra
    Temperature=regression.intercept+regression.slope*Duration*t
    return Temperature
    
#It computer the mean of a region of CPS (greater than a setpoint named lower energy or lower than a setpoint named upper energy) 
#and subtract it to to the data for each spectrum so that all the spectra cross the zero line.
def Normalize(XPS, NumberOfSpectra, NumberOfData):  
    
    x_len=XPS.shape[0]//NumberOfSpectra
    #reshape the data to compute the mean for each spectrum
    Means=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0:NumberOfData],axis=1)

    #subtratct the mean to each spectrum 
    for i in range (0, NumberOfSpectra):
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-= Means[i]
    
    return XPS
    
#For each spectrum, it computes the linear background and subtractit to the data
#To do so, it uses two doublets (x,y),
#they are the mean of the Point number of points at the right of the left edge and at the left of the rigth edge
def RemoveLinearBackgorund(XPS, Points, NumberOfSpectra):
    x_len=XPS.shape[0]//NumberOfSpectra

    for i in range (0, NumberOfSpectra):
        #for each spectrum compute the doublets (x,y) that will be used to calculate the slope m and intercept q
        y1=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i][0:Points])
        y2=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i][x_len-Points:x_len])

        x1=np.mean(XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i][0:Points])
        x2=np.mean(XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i][x_len-Points:x_len])

        #compute slope and intercept
        m=(y2-y1)/(x2-x1)
        q=y2-m*x2

        #subtract to the CPS the as computed linear background
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i] -= m*XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i] + q
        
    return XPS

# Automated code

## Parameters

In [ ]:
#Path to the XPS folder
Path="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/"
#Folder of the Temperature ramp and XPS data
FolderTR=Path+"0009_100321_AQ"
#File of the temperature ramp
FileTR="20210310_TemperatureRamp"

#Files with XPS Data
files=["Br3p","N1s","Au4f"]
#Number of data per spectrum
DataPerSpectra=[409,409,169]
#number of columns comprising the headers of the XPS data files
HeaderColumns=8

## Main

In [ ]:
#Prepare the pandas dataframe with the data
TemperatureRamp = ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

for j in range(0, len(files)):
    
    FileXPS=files[j] #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
    DataPerSpectrum=DataPerSpectra[j] #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
    NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

    #Read the XPS data and assign them to a pandas dataframe
    XPS=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

    #number of spectra composing the dataset
    NumberOfSpectra=(XPS.shape[0]//NumericDataPerSpectrum)
                     
    #prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
    Temperature=TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)
    
    #redefine the length of the vectors along the x axis so that the graph works even when masking the data
    x_len=XPS.shape[0]//NumberOfSpectra
    #Normalize and print the data. ALl the traces should cross the horizintal line in the same point
    XPS=Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)
    #Remove the linear background and print the data
    XPS=RemoveLinearBackgorund(XPS, Points=30, NumberOfSpectra=NumberOfSpectra)
    
    print(FileXPS)
    
    #plot the graph
    fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
    
    fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                      xaxis = dict(showexponent ='none'),
                      margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                      autosize=False, width=600, height=600)

    fig.update_traces(showscale=False) #hides the colorbar
    fig.show()
    #save picture 
    fig.write_image(FolderTR+"/"+FileXPS+".svg")

# Previous versions of the code

## Temperature Ramp

In [ ]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0009_100321_AQ" #Folder of the Temperature ramp
FileTR="20210310_TemperatureRamp" #File of the temperature ramp
#Prepare the pandas dataframe with the data
TemperatureRamp = ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
regression = stats.linregress(TemperatureRamp['Time'], TemperatureRamp['Temperature'])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Time']*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

## XPS Data

In [ ]:
HeaderColumns=8 #number of columns comprising the headers
FileXPS="Au4f" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=169 #number of rows of each spectrum including the headers. 169 for Au, 409 for Br and N
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPS=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPS.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSAu=SetTemperatureColumn(XPSAu, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPS.shape[0]//NumberOfSpectra
#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPS=Normalize(XPS,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)
#Remove the linear background and print the data
XPS=RemoveLinearBackgorund(XPS, Points=30, NumberOfSpectra=NumberOfSpectra)

In [ ]:
fig = go.Figure(data=go.Heatmap(
    z=XPS["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPS["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
#save picture 
#fig.write_image(FolderTR+"/"+FileXPS+".svg") #save picture 

## Code for each peak

### Au

In [ ]:
FileXPS="Au4f" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=169 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSAu=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSAu.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSAu=SetTemperatureColumn(XPSAu, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSAu.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, subplot_titles=("Raw Data", "Normalization", "Background Subtraction"), shared_xaxes=False)
                    #Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSAu=Normalize(XPSAu,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSAu=RemoveLinearBackgorund(XPSAu, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)

fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"Au"+".html")

fig.show()

In [ ]:
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False)#redefine the length of the vectors along the x axis so that the graph works even when masking the data

x_len=XPSAu.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSAu["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)
fig.update_traces(showscale=False) #hides the colorbar
fig.show()
#save picture 
fig.write_image(FolderTR+"/"+"Au.svg") #save picture 

### N

In [ ]:
#FileXPS="BackGroundSubtractionN-Smooth" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
FileXPS="N1s"
DataPerSpectrum=209 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSN=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSN.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSN, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=TemperatureVector(regression, time=TemperatureRamp['Time'], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSN.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

#Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSN=Normalize(XPSN,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSN=RemoveLinearBackgorund(XPSN, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSN["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
    
fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"N"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSN.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSN["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSN["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"N.svg") #save picture 

### Br

In [ ]:
FileXPS="Br3p" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSBr=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSBr.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSBr, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
#Temperature=TemperatureVector(regression, NumberOfSpectra)

In [ ]:
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = make_subplots(rows=3, cols=1, subplot_titles=("Raw Data", "Normalization", "Backrgound Subtraction", "Again Normalized"), shared_xaxes=False)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

XPSBr=Normalize(XPSBr,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

XPSBr=RemoveLinearBackgorund(XPSBr, Points=50, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='CPS')


#fig.write_html(FolderTR+"/"+"Br"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSBr["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"Br.svg") #save picture 

# Leo's Data

## Temperature Ramp

In [ ]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/LeoTPXPS" #Folder of the Temperature ramp
FileTR="Datalog_ZnTPP_Au111_TP_2" #File of the temperature ramp
#Prepare the pandas dataframe with the data
TemperatureRamp = ReadTemperatureData(FolderTR,FileTR)
#Transform the time in hours
AdjustTime(TemperatureRamp, unit=24) #24 means 24 hours
#Fit the temperature data
Mask= TemperatureRamp['Temperature'] <= 250
regression = stats.linregress(TemperatureRamp['Time'][Mask], TemperatureRamp['Temperature'][Mask])

In [ ]:
#plot the temperature ramp, fit and setpoint
fig = go.Figure()
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Temperature'], name="Ramp Data", mode='markers'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'][Mask], y=TemperatureRamp['Time'][Mask]*regression.slope+regression.intercept, name="Ramp Fit", mode='lines'))
fig.add_trace(go.Scatter(x=TemperatureRamp['Time'], y=TemperatureRamp['Setpoint'], name="Setpoint"))
fig.update_layout(xaxis_title='Time (h)', yaxis_title='Temperature (°C)')
fig.show()

## Br

In [ ]:
HeaderColumns=8 #number of columns comprising the headers
FileXPS="LeoBr" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=489 #number of rows of each spectrum including the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSBr=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSBr.shape[0]//NumericDataPerSpectrum)

#Add a column to the dataframe corresponding to the initial temperature of the spectrum
#XPSN=SetTemperatureColumn(XPSBr, NumberOfSpectra, regression)

#prepare a vecture of temperatures for thee heat map. It is as long as the number of spectra and requires the temperature ramp to be computed
Temperature=TemperatureVector(regression, time=TemperatureRamp['Time'][Mask], NumberOfSpectra=NumberOfSpectra)

In [ ]:
x_len=XPSBr.shape[0]//NumberOfSpectra

#Print 3 subplot to show each stage of the manipulation: rwad data, normalization and linear background subtraction
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

#Print the traces of the raw data
for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=1, col=1)

#Normalize and print the data. ALl the traces should cross the horizintal line in the same point
XPSBr=Normalize(XPSBr,NumberOfSpectra=NumberOfSpectra, NumberOfData=50)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=2, col=1)

#Remove the linear background and print the data
XPSBr=RemoveLinearBackgorund(XPSBr, Points=30, NumberOfSpectra=NumberOfSpectra)

for i in range (0, NumberOfSpectra):
    fig.append_trace(go.Scatter(x=XPSBr["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=str(i)),
                     row=3, col=1)
    
fig.update_xaxes(title_text="Binding Energy (eV)", row=3, col=1)
fig.update_yaxes(title_text="CPS", row=1, col=1)
fig.update_yaxes(title_text="CPS", row=2, col=1)
fig.update_yaxes(title_text="CPS", row=3, col=1)
    
#fig.write_html(FolderTR+"/"+"Br"+".html")

fig.show()

In [ ]:
#redefine the length of the vectors along the x axis so that the graph works even when masking the data
x_len=XPSBr.shape[0]//NumberOfSpectra

fig = go.Figure(data=go.Heatmap(
    z=XPSBr["CPS"].values.reshape(NumberOfSpectra,x_len), #reshape the dataframe to form a matrix required for the heatmpa. Each row is a spectrum (CPS)
    x=XPSBr["Binding Energy"][0:x_len], #add the x-axis scale, it must be as long as the spectra
    y=Temperature, #the temperature of the y-axis depends only on the number of spectra
    colorscale= 'Jet'))
fig.update_layout(xaxis_title='Binding Energy (eV)', yaxis_title='Temperature (°C)',
                  xaxis = dict(showexponent ='none'),
                  margin=go.layout.Margin(l=0,r=0,b=0,t=0),
                  autosize=False, width=600, height=600)

fig.update_traces(showscale=False) #hides the colorbar
fig.show()
fig.write_image(FolderTR+"/"+"Br.svg") #save picture 